In [1]:
import pandas as pd
import numpy as np
from pathlib import Path


# --- 1. CONFIGURATION ---
PROJECT_ROOT = Path("/home/onyxia/work/Gestion-portefeuille/")
INPUT_PATH = PROJECT_ROOT / "data/interim/cac40_interim_features.csv"
CLUSTERS_PATH = PROJECT_ROOT / "data/processed/clusters_mapping.csv"
OUTPUT_DIR = PROJECT_ROOT / "data/processed"
OUTPUT_DIR.mkdir(exist_ok=True)


TARGET_HORIZON = 5 # Jours dans le futur


# --- 2. CHARGEMENT ---
df = pd.read_csv(INPUT_PATH, parse_dates=['Date'])
df_clusters = pd.read_csv(CLUSTERS_PATH, index_col='Ticker')
cluster_map = df_clusters['Cluster'].to_dict()


print(f"📥 Données chargées : {df.shape}")


# --- 3. INJECTION DES CLUSTERS ---
df['Cluster'] = df['Ticker'].map(cluster_map)


# Vérifier les tickers sans cluster
missing_clusters = df[df['Cluster'].isna()]['Ticker'].unique()
if len(missing_clusters) > 0:
    print(f"⚠️ {len(missing_clusters)} tickers sans cluster : {missing_clusters.tolist()}")
    df = df.dropna(subset=['Cluster'])


# --- 4. CRÉATION DE LA TARGET ---
df = df.sort_values(['Ticker', 'Date']).reset_index(drop=True)
df['Target_Price'] = df.groupby('Ticker')['Close'].shift(-TARGET_HORIZON)
df['Target'] = (df['Target_Price'] > df['Close']).astype(int)


# --- 5. NETTOYAGE ---
# Supprimer les derniers jours sans Target
df = df.dropna(subset=['Target_Price'])


# Identifier les features
feature_cols = [c for c in df.columns 
                if c not in ['Date', 'Ticker', 'Target_Price', 'Target', 'Cluster', 'Close']]


# Supprimer lignes avec NaN dans les features
initial_len = len(df)
df = df.dropna(subset=feature_cols)
print(f"🧹 Nettoyage : {initial_len - len(df)} lignes supprimées ({100*(1-len(df)/initial_len):.1f}%)")


# --- 6. ANALYSE DE LA TARGET ---
target_dist = df['Target'].value_counts(normalize=True)
print(f"\n📊 Distribution de la Target (Horizon={TARGET_HORIZON}j) :")
print(target_dist)


if target_dist.max() > 0.65:
    print("⚠️ Classes déséquilibrées détectées !")


# --- 7. SÉPARATION TEMPORELLE ---
df = df.sort_values('Date').reset_index(drop=True)

train_end = df['Date'].quantile(0.70)

df_train = df[df['Date'] < train_end]
df_test = df[df['Date'] >= train_end]


# --- 8. SAUVEGARDE ---
df_train.to_csv(OUTPUT_DIR / "train.csv", index=False)
df_test.to_csv(OUTPUT_DIR / "test.csv", index=False)
df.to_csv(OUTPUT_DIR / "cac40_final_superset.csv", index=False)


print(f"\n✅ Datasets sauvegardés :")
print(f"   📁 Train:  {len(df_train):,} lignes ({df_train['Date'].min().date()} → {df_train['Date'].max().date()})")
print(f"   📁 Test:   {len(df_test):,} lignes ({df_test['Date'].min().date()} → {df_test['Date'].max().date()})")
print(f"   📁 Full:   {len(df):,} lignes")


# --- 9. VÉRIFICATIONS FINALES ---
print(f"\n🔍 Features disponibles : {len(feature_cols)}")
print(f"🎯 Target : {df['Target'].nunique()} classes")
print(f"🏷️ Clusters : {df['Cluster'].nunique()} groupes")


📥 Données chargées : (106019, 16)
🧹 Nettoyage : 0 lignes supprimées (0.0%)

📊 Distribution de la Target (Horizon=5j) :
Target
1    0.538718
0    0.461282
Name: proportion, dtype: float64

✅ Datasets sauvegardés :
   📁 Train:  74,061 lignes (2015-01-30 → 2022-09-07)
   📁 Test:   31,768 lignes (2022-09-08 → 2025-12-12)
   📁 Full:   105,829 lignes

🔍 Features disponibles : 13
🎯 Target : 2 classes
🏷️ Clusters : 5 groupes
